In [26]:
import pandas as pd
import folium
import geopy
from geopy.distance import great_circle
import deap

In [22]:
df_final_cities = pd.read_csv('/Users/admin/Desktop/GitHub/new_repos/jingle-jam-2023/notebooks/final_cities.csv', encoding = "iso-8859-1")


In [17]:
display(df_final_cities)

,City,Latitude,Longitude
0,Athens,37.9838,23.7275
1,Cairo,30.0444,31.2357
2,Hialeah,25.8576,-80.2781
3,Lincoln,40.8136,-96.7026
4,Cleveland,41.4993,-81.6944
5,Bangkok,13.7563,100.5018
6,Gilbert,33.3528,-111.7890
7,Corpus Christi,27.8006,-97.3964
8,Osaka,34.6937,135.5022


In [24]:
def calculate_distance(location1, location2):
    return great_circle(location1, location2).mi

name_1 = df_final_cities.iloc[0, 0]
lat_long_1 = df_final_cities.iloc[0, 1], df_final_cities.iloc[0, 2]

name_2 = df_final_cities.iloc[1, 0]
lat_long_2 = df_final_cities.iloc[1, 1], df_final_cities.iloc[1, 2]

distance = round(calculate_distance(lat_long_1, lat_long_2), 0)
print(f"Distance between {name_1} and {name_2}: {distance} miles")

Distance between Athens and Cairo: 696.0 miles
